In [1]:
# Pull season result data from wikipedia and store in a csv file

from io import StringIO
import pandas as pd
import requests
from bs4 import BeautifulSoup
from IPython.display import HTML

In [2]:
SEASON = 1
base_url = f'https://en.wikipedia.org/wiki/Are_You_the_One%3F_(season_{SEASON})'

In [3]:
xpath = '//*[@id="mw-content-text"]/div[1]/table[2]/tbody/tr/td[1]/table'
response = requests.get(base_url)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
table = soup.find('table', attrs={'class':'wikitable'})
# display HTML table in Jupyter notebook
HTML(str(table))

Male cast members[3],Age,Hometown
Adam Kuhn,24,"Sterling, Virginia"
"Andrean ""Dre"" McCoy",23,"Atlanta, Georgia"
Chris Scali,24,"Brooklyn, New York"
Chris Tolleson,23,Virginia Beach
Dillan Ostrom,23,"Finley, North Dakota"
Ethan Diamond,23,"Denver, Colorado"
Joey Dillon,23,"Atlanta, Georgia"
"John ""JJ"" Jacobs",25,"Washington, DC"
Ryan Malaty,24,"Greeley, Colorado"
"Wesley ""Wes"" Buckles",25,"Phoenix, Arizona"


In [13]:
# pull all tables from the page
tables = soup.find_all('table', attrs={'class':'wikitable'}, limit=5)

In [27]:
print(tables[2])

<table class="wikitable" style="text-align:center;">
<tbody><tr>
<th rowspan="2"><span class="nowrap">Guys</span>
</th>
<th colspan="14" style="text-jalign:center;">Ceremony
</th></tr>
<tr>
<th>1</th>
<th>2</th>
<th>3</th>
<th>4</th>
<th>5</th>
<th>6</th>
<th>7</th>
<th>8</th>
<th>9
</th></tr>
<tr>
<th>Adam
</th>
<td>Brittany</td>
<td style="background:salmon">Shanley</td>
<td>Brittany</td>
<td>Amber</td>
<td style="background:salmon">Shanley</td>
<td>Ashleigh</td>
<td style="background:salmon">Shanley</td>
<td style="background:salmon">Shanley</td>
<td style="background:lightgreen">Shanley
</td></tr>
<tr>
<th>Chris S.
</th>
<td>Ashleigh</td>
<td>Simone</td>
<td>Paige</td>
<td>Paige</td>
<td>Simone</td>
<td>Brittany</td>
<td style="background:salmon">Jacy</td>
<td>Ashleigh</td>
<td style="background:lightgreen">Jacy
</td></tr>
<tr>
<th>Chris T.
</th>
<td>Jessica</td>
<td style="background:salmon">Paige</td>
<td>Simone</td>
<td>Ashleigh</td>
<td style="background:salmon">Paige</td>
<td 

In [28]:
tbl2 = tables[2]
rows = tbl2.find_all('tr')

In [29]:
row = rows[2]
row

<tr>
<th>Adam
</th>
<td>Brittany</td>
<td style="background:salmon">Shanley</td>
<td>Brittany</td>
<td>Amber</td>
<td style="background:salmon">Shanley</td>
<td>Ashleigh</td>
<td style="background:salmon">Shanley</td>
<td style="background:salmon">Shanley</td>
<td style="background:lightgreen">Shanley
</td></tr>

In [42]:
# how do i get the style from the row element?
cols = row.find_all(['td', 'th'])
for col in cols:
    print(col.text.strip(), col.attrs.get('style', ''))

Adam 
Brittany 
Shanley background:salmon
Brittany 
Amber 
Shanley background:salmon
Ashleigh 
Shanley background:salmon
Shanley background:salmon
Shanley background:lightgreen


In [17]:
list_data = []
for tbl in tables:
    # parse html table into list of lists
    data = []
    rows = tbl.find_all('tr')
    for row in rows:
        cols = row.find_all(['th', 'td'])
        cols = [ele.text.strip() for ele in cols]
        data.append(cols)

    list_data.append(data)

for ld in list_data:
    for rw in ld:
        print(rw)
    print('------------------')

['Male cast members[3]', 'Age', 'Hometown']
['Adam Kuhn', '24', 'Sterling, Virginia']
['Andrean "Dre" McCoy', '23', 'Atlanta, Georgia']
['Chris Scali', '24', 'Brooklyn, New York']
['Chris Tolleson', '23', 'Virginia Beach']
['Dillan Ostrom', '23', 'Finley, North Dakota']
['Ethan Diamond', '23', 'Denver, Colorado']
['Joey Dillon', '23', 'Atlanta, Georgia']
['John "JJ" Jacobs', '25', 'Washington, DC']
['Ryan Malaty', '24', 'Greeley, Colorado']
['Wesley "Wes" Buckles', '25', 'Phoenix, Arizona']
------------------
['Female cast members[3]', 'Age', 'Hometown']
['Amber Lee', '22', 'Austin, Texas']
['Ashleigh Feaster', '23', 'Akron, Ohio']
['Brittany Baldassari', '24', 'Boston, Massachusetts']
['Coleysia Chestnut', '23', 'Selma, Alabama']
['Jacinda "Jacy" Rodriguez', '24', 'Carolina, Puerto Rico']
['Jessica Perez', '23', 'Miami, Florida']
['Kayla Lusby', '22', 'Greenfield, Tennessee']
['Paige Brendel', '22', 'Phoenix, Arizona']
['Shanley McIntee', '24', 'Mishawaka, Indiana']
['Simone Kelly', '

In [25]:
tbl2 = list_data[2]
ceremony_idx = pd.Index([int(i) for i in tbl2[1]], name='Ceremony')
correct_matches = pd.Series(tbl2[-1][1:], index=ceremony_idx,
                            name='n_correct')

display(correct_matches)
sex = tbl2[0][0][:-1].lower()
header = pd.Index([sex] + ceremony_idx.tolist(), name='ceremony')
df2 = pd.DataFrame(tbl2[2:-1], columns=header)
df2 = (df2
       .set_index(sex)
       .stack()
       .squeeze()
       .rename('match')
       .reset_index())
df2

Ceremony
1     2
2     4
3     2
4     2
5     5
6     5
7     7
8     8
9    10
Name: n_correct, dtype: object

,girl,ceremony,match
0,Amber,1,Ryan
1,Amber,2,Ethan
2,Amber,3,Ethan
3,Amber,4,Adam
4,Amber,5,Ethan
...,...,...,...
85,Simone,5,Chris S.
86,Simone,6,Joey
87,Simone,7,Dre
88,Simone,8,Dre


In [18]:
tbl2 = list_data[2]
idx = pd.MultiIndex.from_product([tbl2[0], tbl2[1]],
                                 names=['Contestant', 'Partner'])
df = pd.DataFrame(tbl2[2:], columns=tbl2[:2])
df

ValueError: all arrays must be same length

In [15]:
from io import StringIO

In [13]:
for tbl in tables:
    display(HTML(str(tbl)))
    print('------------------')

Male cast members[3],Age,Hometown
Adam Kuhn,24,"Sterling, Virginia"
"Andrean ""Dre"" McCoy",23,"Atlanta, Georgia"
Chris Scali,24,"Brooklyn, New York"
Chris Tolleson,23,Virginia Beach
Dillan Ostrom,23,"Finley, North Dakota"
Ethan Diamond,23,"Denver, Colorado"
Joey Dillon,23,"Atlanta, Georgia"
"John ""JJ"" Jacobs",25,"Washington, DC"
Ryan Malaty,24,"Greeley, Colorado"
"Wesley ""Wes"" Buckles",25,"Phoenix, Arizona"


------------------


Female cast members[3],Age,Hometown
Amber Lee,22,"Austin, Texas"
Ashleigh Feaster,23,"Akron, Ohio"
Brittany Baldassari,24,"Boston, Massachusetts"
Coleysia Chestnut,23,"Selma, Alabama"
"Jacinda ""Jacy"" Rodriguez",24,"Carolina, Puerto Rico"
Jessica Perez,23,"Miami, Florida"
Kayla Lusby,22,"Greenfield, Tennessee"
Paige Brendel,22,"Phoenix, Arizona"
Shanley McIntee,24,"Mishawaka, Indiana"
Simone Kelly,23,"Atlanta, Georgia"


------------------


------------------


...

Clue[4][5],Perfect match
1,Ethan & Amber
2,Brittany & Joey


------------------


------------------


Cast member,Seasons of The Challenge
Adam Kuhn,Battle of the Exes II
Brittany Baldassari,Battle of the Exes II
"John ""JJ"" Jacobs",Battle of the Exes II
Simone Kelly,"Battle of the Exes II, Rivals III, XXX: Dirty 30"


------------------


In [6]:
# how to I get the table into a pandas dataframe with just the data and not the urls?
dfs = pd.read_html(StringIO(table))
df.head()

/var/folders/_j/qtktfgps2sgc9s79z5twkgjc0000gn/T/ipykernel_36927/3671702274.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]


,Male cast members[3],Age,Hometown
0,Adam Kuhn,24,"Sterling, Virginia"
1,"Andrean ""Dre"" McCoy",23,"Atlanta, Georgia"
2,Chris Scali,24,"Brooklyn, New York"
3,Chris Tolleson,23,Virginia Beach
4,Dillan Ostrom,23,"Finley, North Dakota"


In [7]:
pd.read_html?

Signature:
pd.read_html(
    io: 'FilePath | ReadBuffer[str]',
    *,
    match: 'str | Pattern' = '.+',
    flavor: 'str | None' = None,
    header: 'int | Sequence[int] | None' = None,
    index_col: 'int | Sequence[int] | None' = None,
    skiprows: 'int | Sequence[int] | slice | None' = None,
    attrs: 'dict[str, str] | None' = None,
    parse_dates: 'bool' = False,
    thousands: 'str | None' = ',',
    encoding: 'str | None' = None,
    decimal: 'str' = '.',
    converters: 'dict | None' = None,
    na_values: 'Iterable[object] | None' = None,
    keep_default_na: 'bool' = True,
    displayed_only: 'bool' = True,
    extract_links: "Literal[None, 'header', 'footer', 'body', 'all']" = None,
    dtype_backend: 'DtypeBackend | lib.NoDefault' = <no_default>,
    storage_options: 'StorageOptions' = None,
) -> 'list[DataFrame]'
Docstring:
Read HTML tables into a ``list`` of ``DataFrame`` objects.

Parameters
----------
io : str, path object, or file-like object
    String, path object